In [1]:
!pip install LangChain -q
!pip install -U langchain-community -q
!pip install pypdf -q
!pip install yt_dlp -q
!pip install pydub -q
!pip install unstructured -q
!pip install docx2txt -q
!pip install openai -q
!pip install python-dotenv -q
!pip install xlrd -q
!pip install tiktoken -q
!pip install chromadb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.9.0 requires keras-core, which is not installed.
keras-nlp 0.12.1 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1 which is incompatible.
jupyterlab 4.2.1 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpys

In [2]:
from openai import OpenAI
import openai
import os
import numpy as np
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
openai.api_key = user_secrets.get_secret("openai_api")
client = OpenAI(
    # This is the default and can be omitted
    api_key=openai.api_key,
)

llm_model = "gpt-3.5-turbo"

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [4]:
sentence1 = "My dog Rover likes to chase squirrels"
sentence2 = "Fluffy my cat, refuses to eat from a can"
sentence3 = "The chevy bolt accelerates to 60 mph in 6.7 seconds"

We can then use the embedding class to create an embedding for each sentence. 



In [5]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [6]:
print('The similarity between sentences one and two is' , np.dot(embedding1, embedding2))
print('The similarity between sentences one and three is' , np.dot(embedding1, embedding3))
print('The similarity between sentences two and three' , np.dot(embedding2, embedding3))

The similarity between sentences one and two is 0.7994217224486891
The similarity between sentences one and three is 0.7492779153935134
The similarity between sentences two and three 0.6991800057325509


In [7]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("/kaggle/input/machine-learning-lectures/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("/kaggle/input/machine-learning-lectures/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("/kaggle/input/machine-learning-lectures/MachineLearning-Lecture02.pdf"),
    PyPDFLoader("/kaggle/input/machine-learning-lectures/MachineLearning-Lecture03.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [8]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

splits = text_splitter.split_documents(docs)
len(splits)

209

In [9]:
from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma/'
!rm -rf ./docs/chroma  # remove old database files if any

In [10]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

print(vectordb._collection.count())

209


In [11]:
question = "What are the names and emails of the course TA?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [12]:
docs[0].page_content

"cs229-qa@cs.stanford.edu. This goes to an acc ount that's read by all the TAs and me. So \nrather than sending us email individually, if you send email to this account, it will \nactually let us get back to you maximally quickly with answers to your questions.  \nIf you're asking questions about homework probl ems, please say in the subject line which \nassignment and which question the email refers to, since that will also help us to route \nyour question to the appropriate TA or to me  appropriately and get the response back to \nyou quickly.  \nLet's see. Skipping ahead — let's see — for homework, one midterm, one open and term \nproject. Notice on the honor code. So one thi ng that I think will help you to succeed and \ndo well in this class and even help you to enjoy this cla ss more is if you form a study \ngroup.  \nSo start looking around where you' re sitting now or at the end of class today, mingle a \nlittle bit and get to know your classmates. I strongly encourage you to f

In [13]:
question = "what did they say about matlab?"
docs = vectordb.similarity_search(question,k=5)


In [14]:
docs[0]


Document(metadata={'page': 8, 'source': '/kaggle/input/machine-learning-lectures/MachineLearning-Lecture01.pdf'}, page_content='those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people call it a free ve rsion of MATLAB, which it sort  of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t s een MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to  learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your  own home computer or something if you \ndon\'t have a MATLAB license, for the purposes of  this class, there\'s also — [inaudible] \nwrite that down [inaudible] MATLAB — there\' s also a software package called Octave \nthat you can download for free off the Inter

In [15]:
docs[1]

Document(metadata={'page': 8, 'source': '/kaggle/input/machine-learning-lectures/MachineLearning-Lecture01.pdf'}, page_content='those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people call it a free ve rsion of MATLAB, which it sort  of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t s een MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to  learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your  own home computer or something if you \ndon\'t have a MATLAB license, for the purposes of  this class, there\'s also — [inaudible] \nwrite that down [inaudible] MATLAB — there\' s also a software package called Octave \nthat you can download for free off the Inter

In [16]:
question = "what did they say about supervised machine learning in the third lecture?"
docs = vectordb.similarity_search(question,k=5)
for doc in docs:
    print(doc.metadata)

{'page': 8, 'source': '/kaggle/input/machine-learning-lectures/MachineLearning-Lecture01.pdf'}
{'page': 8, 'source': '/kaggle/input/machine-learning-lectures/MachineLearning-Lecture01.pdf'}
{'page': 8, 'source': '/kaggle/input/machine-learning-lectures/MachineLearning-Lecture01.pdf'}
{'page': 8, 'source': '/kaggle/input/machine-learning-lectures/MachineLearning-Lecture01.pdf'}
{'page': 2, 'source': '/kaggle/input/machine-learning-lectures/MachineLearning-Lecture01.pdf'}


In [17]:
print(docs[4].page_content)


So in this class, we've tried to convey to you a broad set of principl es and tools that will 
be useful for doing many, many things. And ev ery time I teach this class, I can actually 
very confidently say that af ter December, no matter what yo u're going to do after this 
December when you've sort of completed this  class, you'll find the things you learn in 
this class very useful, and these things will be useful pretty much no matter what you end 
up doing later in your life.  
So I have more logistics to go over later, but let's say a few more words about machine 
learning. I feel that machine learning grew out of  early work in AI, early work in artificial 
intelligence. And over the last — I wanna say last 15 or last 20 years or so, it's been viewed as a sort of growing new capability for computers. And in particular, it turns out 
that there are many programs or there are many applications that you can't program by 
hand.  
For example, if you want to get a computer to read ha